In [1]:

import torchvision
import torchvision.transforms as transforms


In [2]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch

batch_size = 10
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4,
                                           pin_memory=True,
                                           persistent_workers=True)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=4,
                                          pin_memory=True,
                                          persistent_workers=True)

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename="classifier_{epoch:02d}",
    every_n_epochs=5,
    save_top_k=-1,
)

In [5]:

trainer = pl.Trainer(accelerator='gpu', max_epochs=15, callbacks=[checkpoint_callback])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [6]:
from lab4.inception import Inception

#model = Inception(num_classes=10)
model = Inception.load_from_checkpoint('./checkpoints/classifier_epoch=09.ckpt', num_classes=10)

In [7]:
torch.set_float32_matmul_precision('medium')

In [20]:
trainer.fit(model, train_dataloaders=train_loader)


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name        | Type           | Params
------------------------------------------------
0  | conv1       | ConvBlock      | 9.6 K 
1  | conv2       | ConvBlock      | 111 K 
2  | maxpool     | MaxPool2d      | 0     
3  | avgpool     | AvgPool2d      | 0     
4  | dropout     | Dropout        | 0     
5  | linear      | Linear         | 10.2 K
6  | auxiliary4a | Auxiliary      | 2.2 M 
7  | auxiliary4d | Auxiliary      | 2.2 M 
8  | inception3a | InceptionBlock | 164 K 
9  | inception3b | InceptionBlock | 390 K 
10 | inception4a | InceptionBlock | 377 K 
11 | inception4b | InceptionBlock | 450 K 
12 | inception4c | InceptionBlock | 511 K 
13 | inception4d | InceptionBlock | 606 K 
14 | inception4e | InceptionBlock | 870 K 
15 | inception5a | InceptionBlock | 1.0 M 
16 | inception5b | InceptionBlock | 1.4 M 
---------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\azari\AppData\Local\Programs\Python\ml_labs\venv\lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [8]:
trainer.test(model, test_loader)


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      1000
           1       0.94      0.92      0.93      1000
           2       0.73      0.88      0.80      1000
           3       0.75      0.76      0.76      1000
           4       0.91      0.77      0.84      1000
           5       0.83      0.77      0.80      1000
           6       0.89      0.89      0.89      1000
           7       0.94      0.84      0.89      1000
           8       0.90      0.93      0.91      1000
           9       0.88      0.95      0.91      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test_accuracy_epoch    │     0.857200026512146     │
│      test_loss_epoch      │    0.5536240935325623     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.5536240935325623,
  'test_accuracy_epoch': 0.857200026512146}]